In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
import nltk
from IPython.display import Markdown

from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
# from SeaNMF import *

sys.path.insert(0, r'c:\Users\joneh\master_thesis\src_HF')
from utils.main_utils import *

### Import text data

In [ ]:
text_df = pd.read_csv(r'C:\Users\joneh\master_thesis\data\raw_news_stories\EIKON_CRU_NEWS_FULL.csv')
# remove all rows where fullStory is 'error'
text_df = text_df[text_df['fullStory'] != 'error']
# remove all rows where fullStory is float
text_df = text_df[text_df['fullStory'].apply(lambda x: isinstance(x, str))]

display(text_df)

### LDA

In [ ]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = ' '.join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ' '.join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in text_df['fullStory']]

# Creating the term dictionary of our courpus, where every unique term is assigned an index.
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

# Creating the object for LDA model using gensim library
Lda = LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50)

print(ldamodel.print_topics(num_topics=10, num_words=3))

In [ ]:
# print a dataframe with the top word for each topic
topics = ldamodel.print_topics(num_topics=10, num_words=10)

for topic_number in range(10):
    topic = topics[topic_number]
    print(topic[0])
    print(topic[1])



In [ ]:
def print_topics(model, vectorizer, top_n=10):
    for idx, topic in tqdm(enumerate(model.components_)):

        # create dataframe
        df = pd.DataFrame(topic, vectorizer.get_feature_names_out(), columns=["score"])

        # sort by score
        df.sort_values(by="score", ascending=False, inplace=True)

        # display top n words
        display(Markdown(f"### Topic {idx}"))
        display(df.head(8))

# Print the topics
print_topics(ldamodel, vectorizer)